In [47]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.2.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [33]:
import com.google.adk.agents.LlmAgent
import com.google.adk.agents.SequentialAgent
import com.google.adk.agents.ParallelAgent
import com.google.adk.agents.LoopAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.memory.InMemoryMemoryService
import com.google.adk.events.Event
import com.google.adk.tools.FunctionTool
import com.google.adk.tools.GoogleSearchTool

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;


---
## ➰ Section 5: Loop Workflows - The Refinement Cycle

**The Problem: One-Shot Quality**

        All the workflows we've seen so far run from start to finish. The `SequentialAgent` and `ParallelAgent` produce their final output and then stop. This 'one-shot' approach isn't good for tasks that require refinement and quality control. What if the first draft of our story is bad? We have no way to review it and ask for a rewrite.

**The Solution: Iterative Refinement**

When a task needs to be improved through cycles of feedback and revision, you can use a `LoopAgent`. A `LoopAgent` runs a set of sub-agents repeatedly *until a specific condition is met or a maximum number of iterations is reached.* This creates a refinement cycle, allowing the agent system to improve its own work over and over.

**Use Loop when:** Iterative improvement is needed, quality refinement matters, or you need repeated cycles.

To learn more, check out the documentation related to [loop agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/).

**Architecture: Story Writing & Critique Loop**

```
graph TD
        A["Initial Prompt"] -- > B["Writer Agent"]
B -- >|story| C["Critic Agent"]
C -- >|critique| D{"Iteration < Max<br>AND<br>Not Approved?"}
D -- >|Yes| B
        D -- >|No| E["Final Story"]

style B fill:#ccffcc
style C fill:#ffcccc
style D fill:#ffffcc
```

<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

In [34]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [35]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [36]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

### 5.1 Example: Iterative Story Refinement

        Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

In [37]:
val initialWriterAgent = LlmAgent.builder()
    .name("InitialWriterAgent")
    .model(geminiModel)
    .instruction("""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
Output only the story text, with no introduction or explanation.""")
    .outputKey("current_story")
    .build()

println("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [38]:
val criticAgent = LlmAgent.builder()
    .name("CriticAgent")
    .model(geminiModel)
    .instruction("""You are a constructive story critic. Review the story provided below.
Story: {current_story}

Evaluate the story's plot, characters, and pacing.
- If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
- Otherwise, provide 2-3 specific, actionable suggestions for improvement.""")
    .outputKey("critique")
    .build()

println("✅ critic_agent created.")

✅ critic_agent created.


Now, we need a way for the loop to actually stop based on the critic's feedback. The `LoopAgent` itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

1. A simple Python function that the `LoopAgent` understands as an "exit" signal.
2. An agent that can call that function when the right condition is met.

First, you'll define the `exit_loop` function:

In [39]:
object StoryTools {
    /**
     * Call this function ONLY when the critique is 'APPROVED',
     * indicating the story is finished and no more changes are needed.
     */
    fun exitLoop(): Map<String, String> {
        return mapOf(
            "status" to "approved",
            "message" to "Story approved. Exiting refinement loop."
        )
    }
}

println("✅ StoryTools object with exitLoop() function created.")

✅ StoryTools object with exitLoop() function created.


To let an agent call this function, we wrap it in a `FunctionTool`. Then, we create a `RefinerAgent` that has this tool.

👉 **Notice its instructions:** this agent is the "brain" of the loop. It reads the `{critique}` from the `CriticAgent` and decides whether to (1) call the `exit_loop` tool or (2) rewrite the story.

In [40]:
val exitLoopTool = FunctionTool.create(
    StoryTools,
    "exitLoop"
)

val refinerAgent = LlmAgent.builder()
    .name("RefinerAgent")
    .model(geminiModel) // Re-using the model from the first cell
    .instruction("""You are a story refiner. You have a story draft and critique.

    Story Draft: {current_story}
    Critique: {critique}

    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""")
    .outputKey("current_story")
    .tools(listOf(exitLoopTool))
    .build()

println("✅ refiner_agent created.")

✅ refiner_agent created.


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of `max_iterations`:

In [41]:
// The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
val storyRefinementLoop = LoopAgent.builder()
    .name("StoryRefinementLoop")
    .subAgents(listOf(criticAgent, refinerAgent))
    .maxIterations(2) // Prevents infinite loops
    .build()

// The root agent is a SequentialAgent that defines the overall workflow:
// Initial Write -> Refinement Loop.
val rootAgent = SequentialAgent.builder()
    .name("StoryPipeline")
    .subAgents(listOf(initialWriterAgent, storyRefinementLoop))
    .build()

println("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [42]:
var runner: InMemoryRunner = InMemoryRunner(rootAgent, "Critiq Flow")


In [43]:
val prompt = "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"

var userMsg: Content? = Content.fromParts(Part.fromText(prompt))


In [44]:
var session: Session? = runner
    .sessionService()
    .createSession("Critiq Flow", "user")
    .blockingGet()


In [51]:

println("Agent response ->")
var resultText = ""
runBlocking {
    var events = runner.runAsync("user", session.id(), userMsg)
        .asFlow()
        .toList()

    events.forEach {
        // println(event.stringifyContent())
        event -> event
            .content().get()
            .parts().ifPresent { parts:List<Part> -> parts
                .forEach {
                    if (it.text().isPresent) {
                        resultText += it.text().get()
                    }
                }
            }
    }
}

MIME(
    "text/markdown" to resultText
)


Agent response ->


Elias, lighthouse keeper for thirty years, knew the coast like the back of his calloused hand. The granite tower was his world, the rhythmic sweep of the lamp his heartbeat. He’d traded a life of restless wandering for this stoic solitude, a choice he’d never regretted, not until this very moment. The sea had been his only companion, its moods more predictable than human nature. One blustery evening, while meticulously cleaning the Fresnel lens that had been his charge for three decades, his rag snagged on something tucked deep within a crevice. He pulled out a vellum scroll, brittle with age, yet strangely supple, as if preserved by more than just time.

As he unrolled it, a faint, ethereal glow emanated from the markings. It wasn't ink, he realized with a jolt, but lines of shimmering, cool blue light that traced unknown coastlines, dotted with constellations that mirrored no sky he'd ever seen. The air around the scroll seemed to grow warmer, charged with an unseen energy. In the center, a single, pulsating golden star marked a spot far beyond any charted ocean. Elias felt a strange vibration hum through the vellum, a resonant thrum that echoed the sudden, unfamiliar quickening of his own pulse. A shiver, not entirely from the wind, traced its way down his spine. Beneath the glowing star, almost lost in the luminescence, was a symbol – a coiled serpent biting its own tail – a sigil he’d never encountered in his extensive readings of maritime lore, nor in the whispered tales of previous keepers. Who had left this here, and why? The unanswered questions, as potent as the map's strange glow, stirred a forgotten ember of longing for the unknown within him, a feeling he thought had long since been extinguished by the steady rhythm of the waves and the solitary sweep of the lamp.The story is very well-crafted and evocative.

**Plot:** The plot is compelling, centered around the discovery of a mysterious and potentially magical map. The inciting incident is strong and immediately creates questions for the reader. The introduction of the Ouroboros symbol adds a layer of ancient mystery.

**Characters:** Elias is a well-established character. His thirty years of dedication to the lighthouse, his trade-off of wandering for solitude, and his immediate reaction to the map all paint a vivid picture of a man whose predictable world has just been irrevocably shaken. The "forgotten ember of longing for the unknown" is a perfect character beat.

**Pacing:** The pacing is excellent. It begins with a grounded depiction of Elias's life, transitions smoothly into the discovery, and then unfolds the map's mysteries at a pace that builds wonder and a touch of apprehension. The final questions effectively leave the reader wanting more.

The story is well-written and complete as an introduction to a larger narrative.

APPROVEDAPPROVED

---
## Section 6: Summary - Choosing the Right Pattern

### Decision Tree: Which Workflow Pattern?

```
graph TD
        A{"What kind of workflow do you need?"} -- > B["Fixed Pipeline<br>(A → B → C)"];
A -- > C["Concurrent Tasks<br>(Run A, B, C all at once)"];
A -- > D["Iterative Refinement<br>(A ⇆ B)"];
A -- > E["Dynamic Decisions<br>(Let the LLM decide what to do)"];

B -- > B_S["Use <b>SequentialAgent</b>"];
C -- > C_S["Use <b>ParallelAgent</b>"];
D -- > D_S["Use <b>LoopAgent</b>"];
E -- > E_S["Use <b>LLM Orchestrator</b><br>(Agent with other agents as tools)"];

style B_S fill:#f9f,stroke:#333,stroke-width:2px
        style C_S fill:#ccf,stroke:#333,stroke-width:2px
        style D_S fill:#cff,stroke:#333,stroke-width:2px
        style E_S fill:#cfc,stroke:#333,stroke-width:2px
```

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

### Quick Reference Table

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based (sub_agents)** | Dynamic orchestration needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |

---

## ✅ Congratulations! You're Now an Agent Orchestrator

In this notebook, you made the leap from a single agent to a **multi-agent system**.

You saw **why** a team of specialists is easier to build and debug than one "do-it-all" agent. Most importantly, you learned how to be the **director** of that team.

You used `SequentialAgent`, `ParallelAgent`, and `LoopAgent` to create deterministic workflows, and you even used an LLM as a 'manager' to make dynamic decisions. You also mastered the "plumbing" by using `output_key` to pass state between agents and make them collaborative.

**ℹ️ Note: No submission required!**

        This notebook is for your hands-on practice and learning only. You **do not** need to submit it anywhere to complete the course.

### 📚 Learn More

        Refer to the following documentation to learn more:

- [Agents in ADK](https://google.github.io/adk-docs/agents/)
- [Sequential Agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)
- [Parallel Agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/)
- [Loop Agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/)
- [Custom Agents in ADK](https://google.github.io/adk-docs/agents/custom-agents/)

### 🎯 Next Steps

        Ready for the next challenge? Stay tuned for Day 2 notebooks where we'll learn how to create **Custom Functions, use MCP Tools** and manage **Long-Running operations!**